## Song Lyrics Generator
### (작사가 인공지능 만들기)
***
**평가문항**  
>***1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  
> -텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?***  
> 
> ***2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
> -특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?***
> 
> ***3. 텍스트 생성모델이 안정적으로 학습되었는가?  
> -텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?***

***
#### 1. 데이터 읽어오기

In [10]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw) # 텍스트를 라인단위로 끊어서 리스트 형태로 읽어옴

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])
# 특수문자들, 길이가 없는 문장 삭제 필요

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]', 'You ever want something ', "that you know you shouldn't have ", "The more you know you shouldn't have it, ", 'The more you want it ', 'And then one day you get it, ', "It's so good too ", "But it's just like my girl ", "When she's around me ", 'I just feel so good, so good ', 'But right now I just feel cold, so cold ', 'Right down to my bones ', "'Cause ooh... ", "Ain't no sunshine when she's gone ", "It's not warm when she's away ", "Ain't no sunshine when she's gone ", "And she's always gone too long ", 'Anytime she goes away ']


***
#### 2. 데이터 정제하기(데이터 전처리)
- 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기.

In [11]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 텐서플로우
import os

# 문장 토큰화
# 1. Hi,를 막기위해 문장부호 양쪽에 공백 추가
# 2. 대소문자 중복을 막기위해 모든 문자를 소문자로 변환
# 3. 특수문자로 인해 한단어로 인식되어 오류를 일으키는 것을 막기위해 특수문자들 제거

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 터링되는지 확인

<start> this is sample sentence . <end>


In [20]:
# 1차 문장정리 데이터에 <start>와 <end>를 넣어 정리
corpus = []
new_corpus = []
bad_corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
# 깔끔하게 정리된 문장에서 <start>와 <end>를 포함해서 문장길이가 15이하인 부분 삭제하기
for i in range((len(corpus))):
    if len(corpus[i].split(' ')) <=15 :
        new_corpus.append(corpus[i])
    else :
        bad_corpus.append(corpus[i])

print(bad_corpus[0])
print(len(new_corpus))

<start> step right up its circus time and get your moneys worth of love today <end>
156013


In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    # 단어장의 인덱스넘버가 12000개이고 띄어쓰기를 기준으로 토큰을 형성한다.
    # 사전에 없는 단어는 <unk>으로 처리한다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  
        # 전체 단어의 개수 
        # 답으로 활용할 단어의 갯수라고 판단, 7000->12000개로 늘림
        filters= ' ',    
        # 별도로 전처리 로직을 추가할 수 있다.
        # ex ) !"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n
        oov_token="<unk>"  
        # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지 결정
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    # 문장의 최대길이를 조절하여 토큰의 갯수를 조절한다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor.shape)
    return tensor, tokenizer

tensor, tokenizer = tokenize(new_corpus)

(156013, 15)


In [6]:
tensor.shape

(156013, 15)

In [13]:
# 데이터 셋 분리
# 소스문장(<start> blah blah <pad><pad><pad>)
# 타겟문장(blah blah <end> <pad><pad><pad>)
src_input = tensor[:, :-1]  
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

***
#### 3. 평가 데이터셋 분리
- 단어장의 크기는 12000 이상으로 설정.
- 총 데이터의 20%를 평가 데이터셋으로 사용

In [14]:
from sklearn.model_selection import train_test_split
# 총데이터를 학습데이터와 평가용 데이터셋으로 나눔(train_test_split())활용.
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

In [15]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (124810, 14)
Target Train:  (124810, 14)


***
#### 4. 인공지능 만들기
- 10Epoch안에 val_loss를 2.2수준으로 줄이기

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [17]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.30229646e-05,  1.65365869e-04, -1.22482495e-04, ...,
         -8.87530623e-05, -6.59474463e-05, -2.39839756e-05],
        [ 8.00087801e-05,  2.65578565e-04, -2.63377180e-04, ...,
         -2.40562149e-04, -1.70083600e-04, -4.41776174e-05],
        [ 3.03188950e-04,  1.93652100e-04, -3.17938364e-04, ...,
         -4.62011260e-04, -3.63254017e-04, -1.40514414e-04],
        ...,
        [ 1.14587579e-04,  1.41202484e-03, -2.62370359e-05, ...,
         -7.27077771e-04, -6.42314204e-04, -1.17238975e-04],
        [ 7.24197307e-05,  1.17003743e-03,  1.74008237e-04, ...,
         -9.42305080e-04, -6.87676584e-05, -2.44093593e-04],
        [-3.50925002e-05,  8.35430052e-04,  2.73941754e-04, ...,
         -1.14611140e-03,  5.88991388e-04, -3.80493264e-04]],

       [[-1.30229646e-05,  1.65365869e-04, -1.22482495e-04, ...,
         -8.87530623e-05, -6.59474463e-05, -2.39839756e-05],
        [ 7.45294471e-07,  4.49933112e-04, -4

In [18]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
609/609 [==============================] - 99s 163ms/step - loss: 3.4966
Epoch 2/10
609/609 [==============================] - 102s 167ms/step - loss: 2.9979
Epoch 3/10
609/609 [==============================] - 103s 168ms/step - loss: 2.8197
Epoch 4/10
609/609 [==============================] - 102s 167ms/step - loss: 2.6868
Epoch 5/10
609/609 [==============================] - 103s 169ms/step - loss: 2.5729
Epoch 6/10
609/609 [==============================] - 103s 169ms/step - loss: 2.4680
Epoch 7/10
609/609 [==============================] - 107s 176ms/step - loss: 2.3705
Epoch 8/10
609/609 [==============================] - 108s 178ms/step - loss: 2.2791
Epoch 9/10
609/609 [==============================] - 104s 170ms/step - loss: 2.1935
Epoch 10/10
609/609 [==============================] - 104s 170ms/step - loss: 2.1128


In [27]:
results = model.evaluate(enc_val, dec_val, verbose=2)

print(results)

976/976 - 10s - loss: 2.0186
2.0185556411743164


#### **결과**
> **train losdd : 2.1**  
> **validation loss : 2.0186**

In [28]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [33]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20)

'<start> i hate to see your face <end> '

#### **결론**
> **다운받은 Song Lyrics 데이터의 크기 : 187088**  
> **문장 토큰화 결과 (문장부호 공백추가, 모든문자 소문자로 변환, 특수문자 제거 (156013, 15)로 변환하였음.  
> 이를 학습데이터와 검증 데이터로 나누어 학습 후 검증하였음.  
> 학습 시 => train losdd : 2.1  
> 검증결과 => validation loss : 2.0186 로 잘 학습 되었음.  
> 해당 모델로 문장 생성 실시.**  
> **'i hate' 이라는 문장을 시작문자로 줬을 때 'i hate to see your face' 문장 생성.**
***
#### **회고록**
> **작사를 하는 인공지능을 만드는 건 생각보다 재밌었다.  
> 단어뭉치들을 학습해서 문장을 만들다니!  
> 이런 기술들이 발전하고 발전하면 정말 음성으로 대화가 가능한 인공지능이 가능할 수도 있을 것 같다.  
> 그런 기술이 상용화되었을 때 그 기술의 한복판에 내가 있었으면 좋겠다.**